In [5]:
import numpy as np
import dadi
import random
import pickle
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../..')) # this is the ml_dadi dir
from data_manip import generating_data
import math

In [6]:
def generate_data_1d_2epoch_unnorm(n_samples):
    # generate params
    params_dict = {}

    while len(params_dict) < n_samples:
        log_theta = random.random() * 3 + 2
        log_nu = random.random() * 4 - 2
        T = random.random() * 1.9 + 0.1
        params_dict[(log_nu,T)]= log_theta

    print(f'n_samples={len(params_dict)}')

    # designate demographic model, sample size, and extrapolation grid
    func = dadi.Demographics1D.two_epoch
    ns = [20]
    pts_l = [40, 50, 60]
    theta_list = [1]
    # specify param in log scale
    logs = [True, False]

    # generate data
    data = generating_data(
        list(params_dict.keys()), theta_list, func, ns, pts_l, logs)

    # relabel with non-dadi unit
    # make new dict with new labels
    data_dict = {}
    # also need to times fs by unlog theta

    # right now we have two dicts: params_dict and data[0]
    # both have 1000 items and same log_nu, T tuple as keys
    for key in list(params_dict.keys()):
        log_theta = params_dict[key]
        theta = 10**log_theta
        # make a new tuple key with new params and theta
        # this will be our new training label
        # new_key = key + tuple([log_theta])
        # print(new_key)

        # convert nu and T
        new_nu = math.log10(10**key[0] * theta / 4)
        new_T = key[1] * theta / 2
        # new_key = new_nu, new_T, log_theta
        new_key = new_nu, new_T

        # make new fs using unlog theta
        fs = data[0][key]
        scaled_fs = fs * theta

        # store into new data dict
        data_dict[new_key] = scaled_fs

    return data_dict

In [7]:
train_data = generate_data_1d_2epoch_unnorm(1000)
pickle.dump(train_data, open(f'data/1d_2epoch/train_data_1000', 'wb'), 2)

n_samples=1000


In [4]:
test_data = generate_data_1d_2epoch_unnorm(100)
pickle.dump(test_data, open(f'data/1d_2epoch/test_data', 'wb'), 2)

n_samples=100
